<a href="https://colab.research.google.com/github/Kur1sutaru/balaio-de-scripts/blob/add-license-1/Aprendizagem_n%C3%A3o_supervisionada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aprendizagem não supervisionada

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Introdução
Nesse notebook, analisaremos inicialmente alguns dados do **Viroma**, que consiste em um teste metagenômico para identificação de vírus de RNA em amostras clínicas implementado no Hospital Albert Einstein. Após o processamento da amostra no wet lab e obtenção das sequências por NGS, os dados passam por diversas etapas de controle de qualidade e análise, gerando com resultado final um plot semelhante a este:

![](https://www.researchgate.net/publication/340000406/figure/fig1/AS:870415593377825@1584534657597/Krona-plot-representing-an-overview-of-all-viral-sequences-identified-by-Illumina.ppm)

Na versão original, esse é um gráfico interativo, permitindo navegar entre os diferentes táxons e visualizar o número de sequências obtidos em cada um. A partir desse resultado, os especialistas do laboratório observam as espécies mais prevalentes e definem o diagnóstico.

Para maiores informações sobre o escopo desse teste, visite [este post do blog do Varstation](https://varstation.com/pt/blog/artigos/viroma-a-medicina-de-precisao-que-chegou-de-vez-no-diagnostico-e-tratamento-de-doencas-infecciosas-virais/).

# Importação dos dados

Os dados devidamente anonimizados de diversas amostras de Viroma se encontram em um arquivo CSV. Nesse arquivo, cada coluna a partir da terceira representa a abundância relativa de um gênero encontrado nas amostras. Vamos agora importar esse arquivo e fazer algumas análises exploratórias:

In [ ]:
!wget https://dados-anonimizados-viroma.s3-sa-east-1.amazonaws.com/dados_viroma_anonimizados.csv

In [ ]:
df = pd.read_csv("dados_viroma_anonimizados.csv", index_col="ID")
df

Podemos inspecionar quais são os gêneros mais abundantes:

In [ ]:
df.mean().sort_values(ascending=False).head(10).plot(kind="barh")

In [ ]:
df.median().sort_values(ascending=False).head(10).plot(kind="barh")

Temos diversos tipos de amostra no dataset, com predominância de plasma, swab e líquor. Vamos colocar as amostras dos demais materiais na categoria genérica "Outros", para reduzir o número de categorias nessa variável:

In [ ]:
df["Material"].value_counts().plot(kind="barh")
plt.title("Tipos de amostra por material");

In [ ]:
df["Material"].mask(~df["Material"].isin(["Plasma", "Liquor", "Swab"]), "Outros", inplace=True)
df["Material"].value_counts().plot(kind="barh")
plt.title("Tipos de amostra por material");

Temos também muitas amostras nas quais nenhum patógeno foi identificado. Vamos verificar quantos são esses resultados:

In [ ]:
print("Resultados negativos:\t{0:.1%}".format((df["Resultado"]=="Negativo").mean()))

Alguns gêneros desse dataset são da microbiota natural, enquanto outros são patógenos. Os perfis desses dois grupos tendem a ser bem diferentes:

In [ ]:
"""plt.hist(df["Sphingomonas"][df["Material"]=="Plasma"], bins=30)
plt.title("Distribuição da abundância relativa de Sphingomonas\n(bactéria da microbiota normal)")
plt.xlabel("Abundância")
plt.ylabel("Frequência");"""

In [ ]:
plt.hist(df["Orthohepadnavirus"][df["Material"]=="Plasma"], bins=30)
plt.title("Distribuição da abundância relativa de Orthohepadnavirus\n(vírus da hepatite B)")
plt.xlabel("Abundância")
plt.ylabel("Frequência");

# Redução de dimensionalidade com PCA
O método mais clássico de redução de dimensionalidade é o **PCA (Principal Components Analysis)**. Nessa técnica, o objetivo é projetar o dataset em um espaço de dimensão pequena, mas de maneira que seja possível reconstruir o dataset original da maneira mais precisa possível. A figura a seguir, extraída [deste link](https://fspanero.wordpress.com/2009/12/30/analise-de-componente-principais-pca/), ilustra o princípio dessa técnica:

![](https://fspanero.files.wordpress.com/2009/12/pca.jpg)

Ao tomar um número muito pequeno de componentes principais, podemos perder características importantes do dataset. Por outro lado, escolher um número muito grande de componentes é contraproducente, já que vai contra o propósito da redução de dimensionalidade. Uma forma de evitar essa escolha  arbitrária do número de componentes é definir uma proporção de variância explicada, de modo que o próprio algoritmo escolha o número de componentes que seja necessário para capturar as principais características do dataset.

Vamos por isso em prática com a biblioteca **Scikit-Learn**, que contém implementações de diversos algoritmos de ML. Podemos especificar que queremos um número de dimensões suficiente para capturar 95% da variância do dataset:

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(0.95)
princ_comps = pca.fit_transform(np.log10(df.iloc[:,2:]))

Podemos fazer um gráfico para ver a proporção da variância que é alocada em cada componente:

In [ ]:
plt.plot(1+np.arange(pca.n_components_), np.cumsum(pca.explained_variance_ratio_))
plt.plot(1+np.arange(pca.n_components_), pca.explained_variance_ratio_)
plt.xlim(1, pca.n_components_)
plt.ylim(0,1)
plt.xlabel("Número de componentes")
plt.ylabel("Fração de variância explicada")
plt.grid(True)

*Obs.: em geral, é importante que o dataset passe por ume etapa de scaling, onde cada feature é normalizada para que tenham a mesma variabilidade (em geral medida por desvio padrão ou pela diferença entre valor máximo e mínimo). Nesse dataset, essa normalização já foi feita antes por outro método.*

Podemos ver graficamente as duas primeiras componentes principais. Note que PC2 separa razoavelmente as amostras de plasma e de swab.

In [ ]:
plt.figure(figsize=(10,7))
sns.scatterplot(x=princ_comps[:,0], y=princ_comps[:,1], hue=df["Material"])
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("Redução de dimensionalidade com PCA");

Sabemos que as primeiras componentes concentram a maior parte da variabilidade dos dados. Dessa forma, podemos remover a variação populacional normal subtraindo essas componentes, configurando o chamado **PCA denoising**. O que sobra dos dados então é aquilo que é mais incomum nas amostras, e possivelmente de maior interesse clínico. Vamos chamar essa diferença entre um datapoint e sua projeção no hiperplano das primeiras PCs de erro de reconstrução.

In [ ]:
proj = pca.inverse_transform(princ_comps)
err = np.log10(df.iloc[:,2:]) - proj
err

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(err.values.flatten(), bins=50)
plt.axvline(err.max().max(), ls="--", c="k", label="Mínimo e máximo")
plt.axvline(err.min().min(), ls="--", c="k")
plt.axvline(np.percentile(err.values.flatten(), 1), ls="--", c="gray", label="1º e 99º percentis")
plt.axvline(np.percentile(err.values.flatten(), 99), ls="--", c="gray")
plt.xlabel("Erro de reconstrução")
plt.ylabel("Frequência")
plt.title("Distribuição do erro de reconstrução")
plt.legend()
plt.show();

In [ ]:
pd.DataFrame({
    "Gênero mais alterado":err.T.apply(lambda v: v.index[np.argmax(v)]),
    "Erro de reconstrução":err.T.apply(max),
    "Resultado":df["Resultado"]
}).sort_values(by="Erro de reconstrução", ascending=False).head()

In [ ]:
sns.violinplot(
    y=err.max(axis=1).rename("Máximo erro de reconstrução"),
    x=(df["Resultado"]!="Negativo").map({True:"Positivo", False:"Negativo"}));

## Redução de dimensionalidade com outros métodos: Manifold Learning

Implicitamente, o PCA assume que os dados se concentram em torno de um hiperplano, o que torna as projeções lineares que esse método faz muito eficientes. Porém, muitas vezes os dados se aproximam de uma superfície não-linear, que se enquadra no conceito matemático de **manifold** (ou *variedade*, em português). Entre as técnicas de Manifold Learning, estão Isomap e T-SNE.

O Isomap é muito parecido com o PCoA/MDS: é um método que tenta preservar as distâncias entre os pontos do dataset. A diferença (que torna o Isomap geralmente melhor que o PCoA) é que nesse algoritmo as distâncias são calculadas ao longo da manifold, utilizando o grafo de $k$ vizinhos mais próximos. Veja a figura a seguir que ilustra esse processo:

- **A.** Dados no espaço original
- **B.** Rede de vizinhos mais próximos e cálculo de distâncias
- **C.** Espaço de dimensão reduzida, no qual as distâncias euclidianas refletem as distâncias geodésicas

![](https://benalexkeen.com/wp-content/uploads/2017/05/isomap.png)

Já o T-SNE é um método totalmente desenhado para aumentar a separação entre os clusters em um dataset. Esse algoritmo aplica diversas heurísticas baseadas em entidades como a distribuição T-Student e divergência KL. Vale destacar algumas características desse método:
- É um método **estocástico** e propenso a encontrar **mínimos locais**, então executar o algoritmo duas vezes nem sempre dá resultados reprodutíveis.
- Esse método garante que pontos próximos continuem próximos, mas não garante que pontos distantes continuem distantes (ou seja, pode ocorrer a **fusão de clusters**, mas raramente a separação de um cluster).
- Na Bioinformática, esse método foi abraçado pela comunidade de **scRNAseq**, funcionando muito bem para separar células por padrões de expressão gênica. Veja a seguir um exemplo de T-SNE plot nesse tipo de dado:

![](https://miro.medium.com/max/922/1*P7xpVKDJfO9k4of2n5tfoQ.png)

No dataset do viroma, essas técnicas de redução de dimensionalidade não dão resultados muito melhores do que o PCA. Isso depende muito do dataset e da presença de não linearidades nas relações entre as features.

In [ ]:
from sklearn.manifold import Isomap, TSNE

In [ ]:
proj_isomap = Isomap().fit_transform(np.log10(df.iloc[:,2:]))
plt.figure(figsize=(10,7))
sns.scatterplot(x=proj_isomap[:,0], y=proj_isomap[:,1], hue=df["Material"])
plt.title("Redução de dimensionalidade com Isomap");

In [ ]:
proj_tsne = TSNE().fit_transform(np.log10(df.iloc[:,2:]))
plt.figure(figsize=(10,7))
sns.scatterplot(x=proj_tsne[:,0], y=proj_tsne[:,1], hue=df["Material"])
plt.title("Redução de dimensionalidade com T-SNE");

Vamos dar um exemplo mais clássico (porém menos biológico) de aplicação desses métodos. O próprio Scikit-learn tem um dataset de imagens escaneadas de dígitos escritos à mão, em resolução superbaixa (8x8). Cada variável corresponde a um pixel (0=preto, 16=branco). Esse dataset é muito usado para testar algoritmos de classificação (que não são nosso foco agora).

In [ ]:
from sklearn.datasets import load_digits
pixels, numbers = load_digits(return_X_y=True, as_frame=True)

In [ ]:
plt.imshow(pixels.values[5].reshape((8,8)), cmap=plt.cm.gray_r)

In [ ]:
pca = PCA(0.80)
pca.fit(pixels)
transformed = pca.transform(pixels)

plt.figure(figsize=(10,10))
plt.title("Representação 2D com PCA")
sns.scatterplot(x=transformed[:,0], y=transformed[:,1],hue=numbers.astype("category"))

In [ ]:
isomap = Isomap()
transformed = isomap.fit_transform(pixels)

plt.figure(figsize=(10,10))
plt.title("Representação 2D com Isomap")
sns.scatterplot(x=transformed[:,0], y=transformed[:,1],hue=numbers.astype("category"))

In [ ]:
tsne = TSNE()
transformed_digits = tsne.fit_transform(pixels)

plt.figure(figsize=(10,10))
plt.title("Representação 2D com T-SNE")
sns.scatterplot(transformed_digits[:,0], transformed_digits[:,1], hue=numbers.astype("category"))

## Clustering
Uma parte muito importante da aprendizagem não supervisionada são os chamados algoritmos de clustering, que nos permitem agrupar as instâncias com base em similaridade. Como visto na aula teórica, o algoritmo mais simples para essa tarefa é o **K-Means**:

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmeans = KMeans(n_clusters=10)
cluster = kmeans.fit_predict(transformed_digits)
cluster = pd.Series(cluster, dtype="category", name="cluster")
sns.scatterplot(x=transformed_digits[:,0], y=transformed_digits[:,1], hue=cluster)

Podemos ver no heatmap a seguir que os clusters encontrados correspondem aos valores dos dígitos escaneados. Isso é uma demonstração da capacidade desses algoritmos de captar lógica subjacente aos dados: mesmo em um contexto não supervisionado, foi possível encontrar os grupos relevantes nesse dataset.

In [ ]:
sns.clustermap(pd.crosstab(cluster, numbers), cmap="mako", annot=True)

Outro algoritmo interessante é o GMM. Esse algoritmo é um pouco mais flexível que o K-Means (assume clusters elípticos em vez de esféricos) e tem resultados probabilísticos. Isso é relevante pois dá uma ideia da incerteza na associação instância-cluster.

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gmm = GaussianMixture(10)
cluster = gmm.fit_predict(transformed_digits)
cluster = pd.Series(cluster, dtype="category", name="cluster")
plt.figure(figsize=(15,7))
sns.scatterplot(x=transformed_digits[:,0], y=transformed_digits[:,1], hue=cluster)

In [ ]:
gmm.predict_proba(transformed_digits)

In [ ]:
gmm.predict_proba(transformed_digits).max(axis=1)

In [ ]:
plt.figure(figsize=(15,7))
sns.scatterplot(x=transformed_digits[:,0], y=transformed_digits[:,1], hue=gmm.predict_proba(transformed_digits).max(axis=1),
               palette="RdBu")

Note que essa discussão não esgota as possibilidades em análise de clusters: há um grande número de algoritmos disponíveis, cada um com seus prós e contras. A imagem a seguir, extraída da documentação do Scikit-learn, traz os resultados de diversos algoritmos em diversos datasets, mostrando na prática as vantagens e desvantagens de cada um deles:

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cluster_comparison_0011.png)